# Pre-semester Assignment 
- Sequence Classifier
- Author: 楊晴雯
- IKM Lab 111-1 pre-semester assignment
- Date Created: Aug 27, 2022

In [ ]:
'''
功課: 實作一個文字分類器
dataset使用AGNews
並且比較是否用pre-trained word embeddings在表現上的差異
# dataset EDA
# eval metrics: F1, accuracy, confusion_matrix
# https://github.com/aayushs879/AG-News-Sentence-Classification/blob/master/Sentence_Classification.ipynb
# https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-simple-guide-to-text-classification
# https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/
# https://colab.research.google.com/drive/1kvX7XTJEVLmRYT9w37OjWsIl1gCRZL_U?authuser=2#scrollTo=f3ad0878
# https://www.kaggle.com/code/ishandutta/ag-news-classification-lstm
# https://ai.stackexchange.com/questions/9756/lstm-language-model-not-working
# https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset/code
# https://www.google.com/search?q=ag+news+lstm+&biw=1440&bih=789&sxsrf=ALiCzsbkQlp_OPsqUuq3qdEpNLHfeDNz7Q%3A1661960780835&ei=TIIPY9XPMtbf2roPvt6eyAg&ved=0ahUKEwjVpeOvtvH5AhXWr1YBHT6vB4kQ4dUDCA4&uact=5&oq=ag+news+lstm+&gs_lcp=Cgdnd3Mtd2l6EAMyBAgjECc6BwgAEEcQsAM6BQgAEIAEOgYIABAeEBY6CwgAEB4QDxDJAxAWOgoIABCABBCHAhAUOggIABAeEBYQCjoICAAQHhAIEA1KBAhBGABKBAhGGABQ0wNY3A5gqRBoAXABeACAAU2IAesDkgEBN5gBAKABAcgBCMABAQ&sclient=gws-wiz
'''

'\n功課: 實作一個文字分類器\ndataset使用AGNews\n並且比較是否用pre-trained word embeddings在表現上的差異\n# dataset EDA\n# eval metrics: F1, accuracy, confusion_matrix\n# https://github.com/aayushs879/AG-News-Sentence-Classification/blob/master/Sentence_Classification.ipynb\n# https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-simple-guide-to-text-classification\n# https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/\n# https://colab.research.google.com/drive/1kvX7XTJEVLmRYT9w37OjWsIl1gCRZL_U?authuser=2#scrollTo=f3ad0878\n# https://www.kaggle.com/code/ishandutta/ag-news-classification-lstm\n# https://ai.stackexchange.com/questions/9756/lstm-language-model-not-working\n# https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset/code\n# https://www.google.com/search?q=ag+news+lstm+&biw=1440&bih=789&sxsrf=ALiCzsbkQlp_OPsqUuq3qdEpNLHfeDNz7Q%3A1661960780835&ei=TIIPY9XPMtbf2roPvt6eyAg&ved=0ahUKEwjVpeOvtvH5AhXWr1YBHT6vB4kQ4dUDCA4&uact=5&oq=ag+news+lstm+&gs_lc

In [ ]:
!nvidia-smi

Thu Sep  1 14:02:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT = '/content/drive/MyDrive/111-1成大課程/ML/'

In [ ]:
!pip -q install datasets
!pip -q install torch 
!pip -q install Dataset
!pip -q install tqdm 

     |████████████████████████████████| 365 kB 33.8 MB/s 
     |████████████████████████████████| 212 kB 73.2 MB/s 
     |████████████████████████████████| 115 kB 65.1 MB/s 
     |████████████████████████████████| 120 kB 75.3 MB/s 
     |████████████████████████████████| 127 kB 71.4 MB/s 
     |████████████████████████████████| 209 kB 35.9 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 


### Downloading Dataset 
- AG News (AG’s News Corpus) is a subdataset of AG's corpus of news articles constructed by assembling titles and description fields of articles from the 4 largest classes (“World”, “Sports”, “Business”, “Sci/Tech”) of AG’s Corpus. The AG News contains 30,000 training and 1,900 test samples per class.
- Label Mapping: ```“World”: 0,“Sports”: 1,“Business”: 2,“Sci/Tech”: 3```
- Even (Equal) class distribution 
- No need to do train test split 


In [ ]:
from datasets import list_datasets, load_dataset
agnews_dataset = load_dataset('ag_news')

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
agnews_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
from random import randint
r = randint(0, len(agnews_dataset['train']))
agnews_dataset['train'][r]

{'text': 'Parmalat sues ex-auditors Deloitte, Grant Thornton MILAN, Aug 18 (Reuters) - Parmalat sued its former auditors Deloitte  amp; Touche and Grant Thornton on Wednesday, broadening a legal battle to claw back billions of euros from ex-allies the food group says helped drive it into ...',
 'label': 2}

In [ ]:
trainset, testset = agnews_dataset['train'], agnews_dataset['test']

In [ ]:
# average sequence length, token length 
from collections import defaultdict
import numpy as np 

ClassSeqLen = defaultdict(list)
LabelMapping = {0:"World",
                1:"Sports",
                2:"Business",
                3:"Sci/Tech"}

def get_stats(split):
  for i in range(len(split)):
    data = split[i]
    text, label = data['text'], data['label']
    ClassSeqLen[label].append(len(text))
  for k, v in ClassSeqLen.items():
    avg, var = sum(v)/len(v), np.var(v)
    avg, var = round(avg, 2), round(var, 2)
    print(f'  {LabelMapping[k]}: Average: {avg}, Variance: {var}')
print('== Sequence Length Stats by Class: == ')
print('-- train -- ') 
get_stats(trainset)
print('-- test -- ') 
get_stats(testset)

== Sequence Length Stats by Class: == 
-- train -- 
  Business: Average: 241.41, Variance: 4129.48
  Sci/Tech: Average: 237.22, Variance: 6763.04
  Sports: Average: 224.65, Variance: 2550.1
  World: Average: 242.63, Variance: 4048.9
-- test -- 
  Business: Average: 241.33, Variance: 4103.19
  Sci/Tech: Average: 237.23, Variance: 6769.03
  Sports: Average: 224.58, Variance: 2541.67
  World: Average: 242.49, Variance: 4040.47


In [ ]:
# train-test ratio 
len(trainset)/len(testset)

15.789473684210526

## Configuration

In [ ]:
Config = {
    'BATCHSIZE': 64,
    'LR':1e-4,
    'MAXLEN':50,
    'VOCAB_SIZE':None,
    'DATASET':'AGNews', 
    'EPOCHS':10,
    'WEIGHT_DECAY':1e-5,  #default:0
    'DROPOUT':0.2, 
    'CLIP_GRAD':5.0,
    'NUMCHOICE':4,
    'LSTM_HDIM':256, 
    'EMBDIM':32,
}

## Preprocessing & Tokenization

In [ ]:
import pandas as pd
import numpy as np 
import random
import torch 
import os
import torch.nn as nn
import re
import joblib

In [ ]:
'''credit: https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-simple-guide-to-text-classification'''
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
# https://pytorch.org/text/stable/vocab.html#build-vocab-from-iterator
tokenizer = get_tokenizer("basic_english")
def build_vocab(datasets):
    for dataset in datasets:
        for data in dataset:
            text = data['text']
            text = re.sub(r'[~`\-!@#$%^&*():;"{}_/?><\|.,`0-9]',' ',text) # \{the symbol to be escaped} 
            yield tokenizer(text)
vocabs = build_vocab_from_iterator(build_vocab([trainset, testset]), 
                                   specials=["[UNK]", "[PAD]"], max_tokens = Config['VOCAB_SIZE'])
vocabs.set_default_index(vocabs["[UNK]"]) # if not in vocab, set it as [UNK]

In [ ]:
print(f'[UNK]: {vocabs["[UNK]"]}')
print(f'[PAD]: {vocabs["[PAD]"]}')
print(f'Vocab Size: {len(vocabs.get_itos())}')

[UNK]: 0
[PAD]: 1
Vocab Size: 74649


### Frequency-based Vectorizer

- This approach loses text sequence information as it encodes the sequence into a (,vocab_size) sequence. 
- `CountVectorzier, TfidfVectorizer`

### Dense Representation ✅ 
- simply map word into index 

In [ ]:
# skip 
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer(vocabulary = vocabs.get_itos(), tokenizer=tokenizer)
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
X = X.toarray()
X.shape

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1323: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  "Upper case characters found in"


(4, 74649)

In [ ]:
import re
def tokenize(data):
  x, label = data['text'], data['label']
  maxlen = Config['MAXLEN']
  x = x.split(' ')                    # nltk.word_tokenize, spacy (better tokenize)
  x = x[:maxlen]                      # truncation
  tokenized_x = np.ones(maxlen)      # padding: np.ones since [PAD]: 1 
  tokenized_x[:len(x)] = np.array([vocabs[v] for v in x])
  return {'text':tokenized_x, 'label':label}
trainset = trainset.map(tokenize)
testset = testset.map(tokenize)

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

In [ ]:
split = trainset
import random
r = random.randint(0, len(split))
detokenizer = vocabs.get_itos()
# “World”: 0,“Sports”: 1,“Business”: 2,“Sci/Tech”: 3
print(agnews_dataset['train'][r]['label'])
print(agnews_dataset['train'][r]['text'])
print(tokenizer(agnews_dataset['train'][r]['text']))
[detokenizer[int(x)] for x in split[r]['text']]

0
Philippine rebels vow to end conflicts through talks The Philippine rebel group, the Moro Islamic Liberation Front (MILF) Monday reiterated its resolve to end conflict with the government troops through talks.
['philippine', 'rebels', 'vow', 'to', 'end', 'conflicts', 'through', 'talks', 'the', 'philippine', 'rebel', 'group', ',', 'the', 'moro', 'islamic', 'liberation', 'front', '(', 'milf', ')', 'monday', 'reiterated', 'its', 'resolve', 'to', 'end', 'conflict', 'with', 'the', 'government', 'troops', 'through', 'talks', '.']


['[UNK]',
 'rebels',
 'vow',
 'to',
 'end',
 'conflicts',
 'through',
 'talks',
 '[UNK]',
 '[UNK]',
 'rebel',
 '[UNK]',
 'the',
 '[UNK]',
 '[UNK]',
 '[UNK]',
 '[UNK]',
 '[UNK]',
 '[UNK]',
 'reiterated',
 'its',
 'resolve',
 'to',
 'end',
 'conflict',
 'with',
 'the',
 'government',
 'troops',
 'through',
 '[UNK]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[P

In [ ]:
# Dataloader 
def collator(batch):
  # return tensor X and y
  # reminder: keeping int dtype 
  X = np.vstack([x['text'] for x in batch])
  X = torch.from_numpy(X)
  X = X.long()
  y = torch.tensor([x['label'] for x in batch], 
                    dtype=torch.long)
  # print(X.shape, y.shape)
  return X, y
from torch.utils.data import TensorDataset, DataLoader 
trainloader = DataLoader(trainset, shuffle = True, collate_fn = collator, batch_size=Config['BATCHSIZE'])
testloader = DataLoader(testset, shuffle = False, collate_fn = collator, batch_size=Config['BATCHSIZE'])
# each batch: input: (batch_size, maxlen) label: (batch_size)

In [ ]:
for X,y in testloader:
  r = random.randint(0, X.shape[0])
  print([detokenizer[int(x)] for x in X[r,:]])
  print(y[r])
  break

['[UNK]', 'charged', 'in', '[UNK]', 'sexual', 'assault', 'of', '[UNK]', 'girl', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'have', 'arrested', 'a', 'man', 'in', 'the', 'kidnapping', 'and', 'sexual', 'assault', 'of', 'an', '[UNK]', 'girl', 'that', 'frightened', 'this', 'suburban', '[UNK]', 'community', 'last', '[UNK]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

## Building Model 
- sequence classifier (biLSTM) 


In [ ]:
class BiLSTMNet(nn.Module):
  def __init__(self, vocab_size, 
                emb_dim, 
                lstm_hidden_dim,
                is_bidirectional, 
                num_choice, 
                dropout,
                use_pretrained_embeddings = False, 
                frozen_embedding = False, 
                embedding_mat = None):
      super(BiLSTMNet, self).__init__()
      # define variables 
      self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
      self.lstm_hidden_dim = lstm_hidden_dim
      self.emb_dim = emb_dim 
      self.num_directions = 2 if is_bidirectional else 1
      self.num_layers = 2
      self.final_dim = 32

      # define layers
      # !!!!!!!! REVISE padding_idx
      if use_pretrained_embeddings:
        
        self.emb = nn.Embedding.from_pretrained(embedding_mat, 
                                                freeze=frozen_embedding)
      else:
        self.emb = nn.Embedding(vocab_size, emb_dim)
      
      self.lstm = nn.LSTM(emb_dim, lstm_hidden_dim, 
                          self.num_layers, bidirectional= is_bidirectional, 
                          batch_first= True)
      # 先用一層試試看！！
      self.fc1= nn.Linear(self.lstm_hidden_dim*self.num_directions, self.lstm_hidden_dim)
      self.dropout1 = nn.Dropout(dropout)

      # self.fc2 = nn.Linear(self.lstm_hidden_dim, self.final_dim)
      # self.dropout2 = nn.Dropout(dropout) 
      self.activation = nn.ReLU()
      self.classifier = nn.Linear(self.final_dim, num_choice)
      
  def init_hidden(self, bs):
      '''
      the first hidden states needed to be initialized
      bs := batch size of the current batch 
      h0 := first hidden state
      c0 := first cell state
      '''
      # self.lstm_layers * self.num_directions, batch_size, self.hidden_dim (of lstm layer)
      h0 = torch.zeros(self.num_layers * self.num_directions, bs, self.lstm_hidden_dim)
      c0 = torch.zeros(self.num_layers * self.num_directions, bs, self.lstm_hidden_dim)
      # h0 = h0.double()
      # c0 = c0.double()
      h0 = h0.to(self.device)
      c0 = c0.to(self.device)
      return h0, c0

  def forward(self, x, hc0):
      x = self.emb(x) # get the corresponding embeddings
      # print('forward call h:', hidden_states[0].type())
      # print('forward call x:', x.type())
      x, (hn, cn) = self.lstm(x, hc0)  # self.lstm(x, None)
      x = x[:, -1]
      x = self.fc1(x)
      x = self.dropout1(x)
      x = self.fc2(x)
      x = self.dropout2(x)
      x = self.classifier(x)
      x = self.activation(x)
      # print(f'before: {x.shape}')
      # x = x[:, -1]
      # x = x[:,-1,:]
      # print(f'after: {x.shape}')
      return x, (hn, cn)

In [ ]:
model = BiLSTMNet(
    vocab_size = len(vocabs.get_itos()), 
    emb_dim = Config['EMBDIM'], 
    lstm_hidden_dim = Config['LSTM_HDIM'],
    is_bidirectional = False, 
    num_choice = Config['NUMCHOICE'], 
    dropout = Config['DROPOUT'], 
    use_pretrained_embeddings = False, 
    frozen_embedding = False, 
    embedding_mat = None
)

In [ ]:
model

BiLSTMNet(
  (emb): Embedding(74649, 32)
  (lstm): LSTM(32, 256, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=256, out_features=32, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (activation): ReLU()
  (classifier): Linear(in_features=32, out_features=4, bias=True)
)

## Evaluation Metrics

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=vocabs['PAD'])
# Pytorch CEL does softmax for you 
# https://discuss.pytorch.org/t/softmax-cross-entropy-loss/125383
optimizer = torch.optim.AdamW(model.parameters(),
                              weight_decay = Config['WEIGHT_DECAY'], 
                              lr = Config['LR'])
def compute_acc(preds, labels):
    preds = preds.cpu().detach().numpy()
    labels = labels.cpu().detach().numpy()
    preds = np.argmax(preds, axis = 1) # by element 
    return np.sum(preds == labels).item()

## Training [w/o pretrained embeddings]

In [ ]:
!pip install -q wandb

In [ ]:
import wandb
from datetime import datetime
import os
# wandb.login()
# now = datetime.now()
# runname = now.strftime("%m%d-%H")
# model_dir = f'{ROOT}/{runname}'
# os.makedirs(model_dir, exist_ok = True)
# print('* Name of the run:', runname)
# wandb.init(project = "NCKU-IKM ",
#            entity="nana2929", 
#            name = runname, 
#            config = {
#                "batch_size": Config['BATCHSIZE'],
#                "dataset": Config['DATASET'],
#                "learning_rate": Config['LR'],
#                "epochs": Config['EPOCHS'],
#                "dropout": Config['DROPOUT'],
#                "use_pretrained_embeddings": False
#            })

In [ ]:
model = model.to(model.device)

In [ ]:
model

BiLSTMNet(
  (emb): Embedding(74649, 32)
  (lstm): LSTM(32, 256, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=256, out_features=32, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (activation): ReLU()
  (classifier): Linear(in_features=32, out_features=4, bias=True)
)

In [ ]:
# wandb.watch(model)

In [ ]:
from tqdm import tqdm
max_val_acc = float('-inf')
max_grad = Config['CLIP_GRAD']
epochs= Config['EPOCHS']

# 寫accuracy/f1 logging
# TEST
# https://discuss.pytorch.org/t/where-should-i-place-zero-grad/101886
for epoch in range(epochs):
  local_train = {'loss':[], 'acc':[]}
  local_test = {'loss':[], 'acc':[]}
  model.train()
  accurates = 0 
  hidden_states = model.init_hidden(Config['BATCHSIZE'])
  batch_num = 0
  for inputs, labels in tqdm(trainloader):
      batch_num += 1
      inputs, labels = inputs.to(model.device), labels.to(model.device)
      model.zero_grad()
      # avoid backpropagating on the same tensors (-> copying tensors)
      hidden_states = tuple([x.data for x in hidden_states])
      # reuse hidden states  
      outputs, hidden_states = model(inputs, hidden_states)
      
      train_loss = criterion(outputs, labels)
      train_loss.backward()
      accurates += compute_acc(outputs, labels)
      
      nn.utils.clip_grad_norm_(model.parameters(), max_grad) # sequence models 所需
      optimizer.step() 
  model.eval()
  # with torch.no_grad():
  for test_inputs, test_labels in tqdm(testloader):
      test_inputs, test_labels = test_inputs.to(model.device), test_labels.to(model.device)  
      test_outputs, _ = model(inputs, hidden_states)
      # accuracy = compute_acc(outputs, labels)
      val_loss = criterion(test_outputs, test_labels)
  print(f'[LOSS] train loss: {train_loss}, val loss: {val_loss}')
  print(f'EPOCH {epoch+1}|train accuracy: {accurates/len(trainset)}')

100%|██████████| 38/38 [00:01<00:00, 31.86it/s]


[LOSS] train loss: 1.1280851364135742, val loss: 1.1019943952560425
EPOCH 1|train accuracy: 0.24894166666666667


100%|██████████| 38/38 [00:01<00:00, 21.30it/s]


[LOSS] train loss: 1.1102941036224365, val loss: 1.1028625965118408
EPOCH 2|train accuracy: 0.25078333333333336


100%|██████████| 38/38 [00:01<00:00, 32.11it/s]


[LOSS] train loss: 1.093716025352478, val loss: 1.1014388799667358
EPOCH 3|train accuracy: 0.2518916666666667


 75%|███████▌  | 451/600 [00:37<00:12, 12.10it/s]


KeyboardInterrupt: ignored